In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [11]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [12]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [13]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [14]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [15]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([111963, 118584,  84684, 103126])
inputs : 
tensor([[ 1, 37, 68, 74, 67, 73, 54, 62],
        [55, 58,  1, 69, 58, 71, 59, 58],
        [54, 73, 58, 57,  1, 69, 68, 72],
        [58,  1, 55, 65, 68, 68, 57,  1]], device='cuda:0')
targets : 
tensor([[37, 68, 74, 67, 73, 54, 62, 67],
        [58,  1, 69, 58, 71, 59, 58, 56],
        [73, 58, 57,  1, 69, 68, 72, 62],
        [ 1, 55, 65, 68, 68, 57,  1, 59]], device='cuda:0')


### Making the Bigram Language model

In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


I!6y1yQ'Mz2iZ
V1EfH(h?dg"]KjVOQV l41(a-;WKJwRsVaX[0eIWvz4Bg l.J)&;QTVS4sip(Sn__i*G65pv8&4sFhZp65B4iF ZAj']nf&_;9zCNJXQ)?SkCG:Y.,AHyCumWudgg"?f'h?4O,sBgVH0&zF﻿hLBY9h?tXZ
nSw[wNx* !nUZzv:6!sB,U0BsA-RKs;'Csd3-1.jV?pvq4mVzx&-l!3ux8J2(-,28AV_"nB!7*﻿UrzE Tpv0i(Z[wRZei3s;26SogCHTB!_40cJMck1Fu_PcF'Cl8__lrAPZi:nywLqqcW(QMOqa?bw.cwRT2iZq2S!IGtIm'E4;l)p
y!Aq9l3g*Qn'&.LyY2!szF1nBuE9ywK1J[l3or* kpc3"76X I.nY92y 2"S(SmW nr2Re9p_pvH]mV_OQQ﻿Mjyh_Z9[m'Rjl:APAWI;WQ9:hHAAe2iZ65Gfa!EQi:"n]T5Q3j57T"R"']mn;l.bTC4M(;*


In [17]:
#create a AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):

    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 55112, 129338,  85097,  49319])
4.997245788574219
tensor([79660, 81485, 99736, 28315])
4.992439270019531
tensor([102579,  68394, 117867,  88959])
4.933568000793457
tensor([ 76147,  99875, 151407, 175595])
5.088196754455566
tensor([ 31200, 180627,  21795, 119674])
4.935286045074463
tensor([131357, 164146, 121611,  74930])
5.222865104675293
tensor([167390, 140957,  40600,  62014])
4.894834041595459
tensor([170148, 129314, 183277,  40740])
5.102747440338135
tensor([101415,  42299, 100703,  59659])
5.198853969573975
tensor([ 91573,  76343, 104644, 116865])
4.790302276611328
tensor([ 22592,  31031,  70045, 128505])
5.366444110870361
tensor([ 65152,   7792, 139494, 122017])
5.207705974578857
tensor([180203, 174360,  22265, 102875])
5.082995891571045
tensor([ 24157,  62505,  61361, 107817])
4.824490547180176
tensor([141474, 111983,   4305, 111516])
4.995925426483154
tensor([  6437, 129171,  10991,  81179])
4.722561836242676
tensor([ 82607,  49886, 102137,  23189])
5.036575794219971
t

In [18]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


hEg awIEW﻿?tYKj﻿qj; yat6﻿uneAle_1.U9fOqBg&3rzx8B8*vx_tNXmqcFjMOF
am[wighuem"WIF*HA-P1j--zFJ3:;;lm
t!fcabayo?lvchas;O4AXpvawRCl0ep_OqTW"CPloras)rd t!ib1q,J.Othycs brure2winck8RM_Ir gsupnclyo?K[﻿65H;
RT3,ut [wa9,jRsm
anJw
T*gvZ!p&;W(cL.
CsB﻿?ps!Icps JZ:q
potJNWIABg t!Ze.4Y32(rut:"YRBawQWnei),)?"
UCu6FqFq4MXfIK[w.:x''jOxrl!,"(racq2en3rel
-tL:wY-Xmyt)VOj,y,
5. .bD iZn RPyim0ifbXy)b7wc?FSoFvI,UB?p
vZiZeppp,54WMloratbwn bl-P_GUXQPgh t_gb﻿ckX[wTd'CxM"m.iquO.T;;Bu7ws o u,KM!Z9f-&NTCRxR3l el9*46gususe!Z 
